In [1]:
import numpy as np
import pandas as pd
import basedosdados as bd
from datetime import datetime

## Dados necessários

In [2]:
query = '''
  SELECT * 
  FROM `datario.administracao_servicos_publicos.chamado_1746`
  WHERE DATETIME_TRUNC(data_inicio, DAY) = '2023-04-01T00:00:00'
'''
chamado_01042023_df = bd.read_sql(query, billing_project_id = 'projeto-escritorio-dados-rj')

Downloading: 100%|██████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 139.18rows/s]


In [3]:
query = '''
  SELECT * 
  FROM `datario.administracao_servicos_publicos.chamado_1746`
  WHERE data_inicio BETWEEN '2022-01-01T00:00:00' AND '2023-12-31T23:59:59'
  AND subtipo = 'Perturbação do sossego'
'''
chamado_2022_2023_df = bd.read_sql(query, billing_project_id = 'projeto-escritorio-dados-rj')

chamado_2022_2023_df['data_inicio'] = pd.to_datetime(chamado_2022_2023_df['data_inicio']).dt.date

Downloading: 100%|███████████████████████████████████████████████████████████| 42408/42408 [00:25<00:00, 1653.08rows/s]


In [4]:
query = '''
  SELECT * 
  FROM `datario.dados_mestres.bairro`
'''
bairros_df = bd.read_sql(query, billing_project_id = 'projeto-escritorio-dados-rj')

Downloading: 100%|█████████████████████████████████████████████████████████████████| 164/164 [00:11<00:00, 14.26rows/s]


In [5]:
query = '''
  SELECT * 
  FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
'''
ocupacao_eventos_df = bd.read_sql(query, billing_project_id = 'projeto-escritorio-dados-rj')

ocupacao_eventos_df['data_inicial'] = pd.to_datetime(ocupacao_eventos_df['data_inicial']).dt.date
ocupacao_eventos_df['data_final'] = pd.to_datetime(ocupacao_eventos_df['data_final']).dt.date

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 10.07rows/s]


## Questão 1

In [6]:
print(f'Foram abertos {len(chamado_01042023_df)} chamdos no dia 01/04/2023.')

Foram abertos 73 chamdos no dia 01/04/2023.


## Questão 2

In [7]:
tipo_frequente_chamado = chamado_01042023_df["tipo"].value_counts().idxmax()

print(f'O tipo de chamado mais frequente no dia 01/04/2023 é {tipo_frequente_chamado}.')

O tipo de chamado mais frequente no dia 01/04/2023 é Poluição sonora.


## Questão 3

In [8]:
chamados_bairro = chamado_01042023_df.merge(bairros_df, on = 'id_bairro', how = 'left')
chamados_bairros_top3 = chamados_bairro['nome'].value_counts()[:3].index.tolist()

print(f'Os bairros com mais chamados no dia 01/04/2023 são {chamados_bairros_top3[0]}, \
{chamados_bairros_top3[1]} e {chamados_bairros_top3[2]}.')

Os bairros com mais chamados no dia 01/04/2023 são Engenho de Dentro, Leblon e Campo Grande.


## Questão 4

In [9]:
chamados_subprefeitura = chamados_bairro['subprefeitura'].value_counts().idxmax()

print(f'A subprefeitura com mais chamados abertos no dia 01/04/2023 é a subprefeitura {chamados_subprefeitura}.')

A subprefeitura com mais chamados abertos no dia 01/04/2023 é a subprefeitura Zona Norte.


## Questão 5

In [10]:
chamados_bairro[(chamados_bairro['id_chamado'].isnull()) | (chamados_bairro['subprefeitura'].isnull())]

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry_y
0,18516246,2023-04-01 00:55:38,2023-04-01 00:55:38,None,None,None,<NA>,1706,TR/SUBOP/CFT - Coordenadoria de Fiscalização e...,SMTR - Secretaria Municipal de Transportes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Questão 6

In [11]:
print(f'Foram abertos {len(chamado_2022_2023_df)} chamdos desde 01/01/2022 até 31/12/2023.')

Foram abertos 42408 chamdos desde 01/01/2022 até 31/12/2023.


## Questão 7

In [12]:
chamado_2022_2023_df['constante'] = True
ocupacao_eventos_df['constante'] = True

chamado_eventos_join_df = pd.merge(chamado_2022_2023_df, ocupacao_eventos_df, on = 'constante')
chamado_eventos_join_df = chamado_eventos_join_df[(chamado_eventos_join_df['data_inicio'] <= chamado_eventos_join_df['data_final']) & (chamado_eventos_join_df['data_inicio'] >= chamado_eventos_join_df['data_inicial'])]
chamado_eventos_join_df

,id_chamado,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,id_unidade_organizacional_mae,...,justificativa_status,reclamacoes,data_particao,geometry,constante,ano,data_inicial,data_final,evento,taxa_ocupacao
84801,18078140,2022-12-30,2022-12-30 22:03:07,23,2,65615,93,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2022-12-01,POINT(-43.1691884 -22.962173),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
84841,18077988,2022-12-30,2022-12-30 22:14:21,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2022-12-01,POINT(-43.236608 -22.917704),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
84845,18077970,2022-12-30,2022-12-30 22:13:06,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2022-12-01,POINT(-43.236608 -22.917704),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
84857,18077984,2022-12-30,2022-12-30 22:13:46,35,2,65805,2,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2022-12-01,POINT(-43.236608 -22.917704),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
85205,18078946,2022-12-31,2022-12-31 14:22:50,141,5,10629,640,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2022-12-01,None,True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167677,18080335,2023-01-01,2023-01-02 02:30:13,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
167681,18080337,2023-01-01,2023-01-02 02:24:31,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
167685,18080341,2023-01-01,2023-01-02 02:18:39,64,3,81034,232,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,0,2023-01-01,POINT(-43.2867915 -22.8912784),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
167713,18079926,2023-01-01,2023-01-01 21:24:33,42,3,19117,77,70,GM-RIO - Guarda Municipal do Rio de Janeiro,GM-RIO - Guarda Municipal do Rio de Janeiro,...,None,2,2023-01-01,POINT(-43.2679728 -22.8477888),True,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251


## Questão 8

In [13]:
chamados_por_evento = chamado_eventos_join_df['evento'].value_counts().reset_index()
chamados_por_evento.rename(columns={'evento': 'contagem', 'index': 'evento'}, inplace = True)
chamados_por_evento

,evento,contagem
0,Rock in Rio,834
1,Carnaval,241
2,Reveillon,137


In [14]:
print(f'No {chamados_por_evento.loc[0][0]}, foram abertos {chamados_por_evento.loc[0][1]} chamados.')
print(f'No {chamados_por_evento.loc[1][0]}, foram abertos {chamados_por_evento.loc[1][1]} chamados.')
print(f'No {chamados_por_evento.loc[2][0]}, foram abertos {chamados_por_evento.loc[2][1]} chamados.')

No Rock in Rio, foram abertos 834 chamados.
No Carnaval, foram abertos 241 chamados.
No Reveillon, foram abertos 137 chamados.


## Questão 9

In [15]:
ocupacao_eventos_df['duracao'] = (ocupacao_eventos_df['data_final'] - ocupacao_eventos_df['data_inicial'])/ pd.Timedelta(1, unit='d') + 1

duracao_eventos = ocupacao_eventos_df.groupby(['evento']) \
    .agg(duracao = ('duracao', 'sum')) \
    .reset_index()

In [16]:
media_chamados_evento = pd.merge(duracao_eventos, chamados_por_evento, on = 'evento')
media_chamados_evento['media_chamados'] = media_chamados_evento['contagem']/media_chamados_evento['duracao']
media_chamados_evento = media_chamados_evento[['evento', 'media_chamados']]
media_chamados_evento

,evento,media_chamados
0,Carnaval,60.250000
1,Reveillon,45.666667
2,Rock in Rio,119.142857


In [17]:
evento_maior_media_diaria = media_chamados_evento.loc[media_chamados_evento['media_chamados'].idxmax()][0]
print(f'O {evento_maior_media_diaria} teve a maior média diária de chamados.')

O Rock in Rio teve a maior média diária de chamados.


## Questão 10

In [18]:
media_anual = len(chamado_2022_2023_df)/(datetime(2024, 1, 1) - datetime(2022, 1, 1)).days

nova_linha_media_anual = {'evento': 'Ano 2022 e 2023', 
                          'media_chamados': media_anual}

media_chamados_evento = media_chamados_evento.append(nova_linha_media_anual, ignore_index = True)
media_chamados_evento

,evento,media_chamados
0,Carnaval,60.250000
1,Reveillon,45.666667
2,Rock in Rio,119.142857
3,Ano 2022 e 2023,58.093151
